In [1]:
import biogeme.version as ver

In [2]:
print(ver.getText())

biogeme 3.2.12 [2023-11-25]
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import pandas as pd
import biogeme.biogeme as bio
import biogeme.database as db
from biogeme import models
from biogeme.expressions import Beta
import numpy as np

In [4]:
df=pd.read_csv('Dataset 1.csv')#读取表格数据
df

,HHID,ORIGIN_OF_TRIP,DESTINATION_OF_TRIP,TT_WALK,WALKABILITY_INDEX,TT_CYCLE,BIKEABILITY_INDEX,TT_PT,FARE_PT,WT_PT,...,CC_CAR,INCOME,GENDER,AGE,CAR_OWNERSHIP,CHOSEN_MODE,AV_WALK,AV_CYCLE,AV_PT,AV_CAR
0,1,3,7,27,97,35,63,46,7,19,...,2,86,0,23,0,2,1,1,1,1
1,2,8,1,31,100,31,40,27,2,18,...,9,56,0,36,1,3,1,1,1,1
2,3,9,4,52,14,13,22,29,2,5,...,4,59,1,46,0,2,1,1,1,1
3,4,8,6,33,28,16,70,32,2,9,...,2,29,1,27,0,3,1,1,1,1
4,5,2,2,55,11,20,96,51,3,18,...,6,64,0,49,1,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,3,5,52,39,23,41,13,6,14,...,5,13,1,46,0,2,1,1,1,1
9996,9997,6,7,41,3,12,15,34,3,12,...,7,61,1,68,0,4,1,1,1,1
9997,9998,7,5,55,32,41,67,41,1,2,...,3,45,1,43,1,2,1,1,1,1
9998,9999,4,9,19,77,36,90,55,2,6,...,9,25,1,26,0,1,1,1,1,1


In [5]:
df['TC_CAR'] = df['FC_CAR']+df['PC_CAR']+df['CC_CAR']
df['TC_PT'] = df['FARE_PT']+df['AC_PT']
df['JT_PT'] = df['TT_PT']+df['WT_PT']+df['AT_PT']
df.head()

,HHID,ORIGIN_OF_TRIP,DESTINATION_OF_TRIP,TT_WALK,WALKABILITY_INDEX,TT_CYCLE,BIKEABILITY_INDEX,TT_PT,FARE_PT,WT_PT,...,AGE,CAR_OWNERSHIP,CHOSEN_MODE,AV_WALK,AV_CYCLE,AV_PT,AV_CAR,TC_CAR,TC_PT,JT_PT
0,1,3,7,27,97,35,63,46,7,19,...,23,0,2,1,1,1,1,4,12,84
1,2,8,1,31,100,31,40,27,2,18,...,36,1,3,1,1,1,1,16,3,59
2,3,9,4,52,14,13,22,29,2,5,...,46,0,2,1,1,1,1,9,6,44
3,4,8,6,33,28,16,70,32,2,9,...,27,0,3,1,1,1,1,5,2,58
4,5,2,2,55,11,20,96,51,3,18,...,49,1,2,1,1,1,1,10,3,82


In [6]:
# convert pandas dataframe object to Biogeme Database
database = db.Database('application', df)

In [7]:
# display database info
display(database.getSampleSize())
display(database.variables)

10000

{'HHID': HHID,
 'ORIGIN_OF_TRIP': ORIGIN_OF_TRIP,
 'DESTINATION_OF_TRIP': DESTINATION_OF_TRIP,
 'TT_WALK': TT_WALK,
 'WALKABILITY_INDEX': WALKABILITY_INDEX,
 'TT_CYCLE': TT_CYCLE,
 'BIKEABILITY_INDEX': BIKEABILITY_INDEX,
 'TT_PT': TT_PT,
 'FARE_PT': FARE_PT,
 'WT_PT': WT_PT,
 'AT_PT': AT_PT,
 'AC_PT': AC_PT,
 'TT_CAR': TT_CAR,
 'FC_CAR': FC_CAR,
 'PC_CAR': PC_CAR,
 'CC_CAR': CC_CAR,
 'INCOME': INCOME,
 'GENDER': GENDER,
 'AGE': AGE,
 'CAR_OWNERSHIP': CAR_OWNERSHIP,
 'CHOSEN_MODE': CHOSEN_MODE,
 'AV_WALK': AV_WALK,
 'AV_CYCLE': AV_CYCLE,
 'AV_PT': AV_PT,
 'AV_CAR': AV_CAR,
 'TC_CAR': TC_CAR,
 'TC_PT': TC_PT,
 'JT_PT': JT_PT}

In [8]:
# The following statement allows you to use the names of the
# variable as Python variable.
# specifiying the initial values 
globals().update(database.variables)

In [9]:
# Parameters to be estimated
# Arguments:
#   - 1  Name for report; Typically, the same as the variable.
#   - 2  Starting value.
#   - 3  Lower bound.
#   - 4  Upper bound.
#   - 5  0: estimate the parameter, 1: keep it fixed.

ASC_CAR = Beta('ASC_CAR', 0, None, None, 1) 
# 0 is the start value, 1 indicates whrther biversion is fixed, none is upper and lower limits
ASC_WALK= Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE= Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT= Beta('ASC_PT', 0, None, None, 0)
B_TT_CAR = Beta('B_TT_CAR', 0, None, None, 0)
B_TT_WALK = Beta('B_TT_WALK', 0, None, None, 0)
B_TT_BIKE = Beta('B_TT_BIKE', 0, None, None, 0)
B_TC_CAR = Beta('B_TC_CAR', 0, None, None, 0)
B_JT_PT = Beta('B_JT_PT', 0, None, None, 0)
B_TC_PT = Beta('B_TC_PT', 0, None, None, 0)

B_WALKBILITY = Beta('B_WALKBILITY', 0, None, None, 0)
B_BIKEBILITY = Beta('B_BIKEBILITY', 0, None, None, 0)

B_INCOME = Beta('B_INCOME', 0, None, None, 0)
B_GENDER = Beta('B_GENDER', 0, None, None, 0)
B_AGE = Beta('B_AGE', 0, None, None, 0)
B_CAROWNERSHIP = Beta('B_CAROWNERSHIP', 0, None, None, 0)



In [10]:
# Definition of Utility functions
V1 = ASC_WALK  +  B_TT_WALK*TT_WALK+B_WALKBILITY*WALKABILITY_INDEX+B_GENDER*GENDER+B_AGE*AGE+B_CAROWNERSHIP*CAR_OWNERSHIP+B_INCOME*INCOME
V2 = ASC_BIKE  +  B_TT_BIKE*TT_CYCLE+B_BIKEBILITY*BIKEABILITY_INDEX+B_GENDER*GENDER+B_AGE*AGE+B_CAROWNERSHIP*CAR_OWNERSHIP+B_INCOME*INCOME
V3 = ASC_PT +  B_JT_PT*JT_PT + B_TC_PT*TC_PT+B_GENDER*GENDER+B_AGE*AGE+B_CAROWNERSHIP*CAR_OWNERSHIP+B_INCOME*INCOME
V4 = ASC_CAR +  B_TT_CAR*TT_CAR + B_TC_CAR*TC_CAR+B_GENDER*GENDER+B_AGE*AGE+B_CAROWNERSHIP*CAR_OWNERSHIP+B_INCOME*INCOME

In [11]:
# Associate utility functions with the numbering of alternatives
V = {1: V1, 2: V2,3: V3,4: V4}
# v1 v2 ... are the chosen traveling mode

In [12]:
# Associate the availability conditions with the alternatives

av = {1:AV_WALK,2:AV_CYCLE,3:AV_PT,4:AV_CAR}

In [13]:
# The choice model is a logit, with availability conditions
logprob = models.loglogit(V, av,CHOSEN_MODE)

In [14]:
# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'Multilogit_model'

In [15]:
# Estimate the parameters
results = biogeme.estimate()
results.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_BIKE,-2.261157e+00,1.594696e-01,-1.417924e+01,0.000000e+00
ASC_PT,-7.594972e-03,3.247785e-01,-2.338509e-02,9.813431e-01
ASC_WALK,-2.318760e+00,1.567828e-01,-1.478963e+01,0.000000e+00
B_AGE,5.680315e-14,1.797693e+308,3.162020e-322,1.000000e+00
B_BIKEBILITY,5.637616e-03,8.496808e-04,6.634981e+00,3.245448e-11
B_CAROWNERSHIP,1.406192e-16,1.797693e+308,0.000000e+00,1.000000e+00
B_GENDER,4.932078e-17,2.320177e-14,2.125733e-03,9.983039e-01
B_INCOME,-1.890979e-13,2.441524e-15,-7.745077e+01,0.000000e+00
B_JT_PT,-6.340500e-02,4.439129e-03,-1.428321e+01,0.000000e+00
B_TC_CAR,-1.913765e-01,9.764949e-03,-1.959831e+01,0.000000e+00
